In [1]:
import pandas as pd
import numpy as np

In [2]:
# filepaths
all_data = "EDA_data_07_01_2024.csv"
wet_data = "wet_EDA_data_07_01_24.csv"
dry_data = "dry_EDA_data_07_01_24.csv"

output_fp = "../output/convert/"

In [7]:
df = pd.read_csv("../output/extract/" + all_data,header=[0, 1, 2, 3, 4, 5, 6], index_col=0, parse_dates=True)
wet_df = pd.read_csv("../output/extract/" + wet_data, header=[0, 1, 2, 3, 4, 5, 6], index_col=0, parse_dates=True)
dry_df = pd.read_csv("../output/extract/" + dry_data, header=[0, 1, 2, 3, 4, 5, 6], index_col=0, parse_dates=True)

In [ ]:
def convert_cfs_to_taf(df):
    date_column = df.index
    months = date_column.strftime('%m')
    years = date_column.strftime('%Y')

    days_in_month = np.zeros(len(df))

    # Compute the number of days in each month, considering leap years for February
    for i in range(len(months)):
        if months[i] in {"01", "03", "05", "07", "08", "10", "12"}:
            days_in_month[i] = 31
        elif months[i] == "02":
            year = int(years[i])
            if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):
                days_in_month[i] = 29
            else:
                days_in_month[i] = 28
        elif months[i] in {"04", "06", "09", "11"}:
            days_in_month[i] = 30

    columns_to_convert = [col for col in df.columns if ('DEL' in col[1] or 'NDO' in col[1] or 'D_TOTAL' in col[1]) and 'CFS' in col[6]]
    new_columns_dict = {}

    for column in columns_to_convert:
        new_values = df[column].values * 2.29568e-5 * 86400 * days_in_month / 1000
        new_column_name = list(column)
        new_column_name[1] = new_column_name[1] + '_TAF'
        new_column_name[6] = 'TAF'
        new_column_name = tuple(new_column_name)
        new_columns_dict[new_column_name] = new_values

    for new_col, new_values in new_columns_dict.items():
        df[new_col] = new_values

    return df

In [ ]:
df = convert_cfs_to_taf(df)

In [ ]:
wet_df = convert_cfs_to_taf(wet_df)

In [ ]:
dry_df = convert_cfs_to_taf(dry_df)

In [ ]:
df.to_csv(output_fp + "convert_" + all_data, header=True, index=True)
wet_df.to_csv(output_fp + "convert_" + wet_data, header=True, index=True)
dry_df.to_csv(output_fp + "convert_" + dry_data, header=True, index=True)